## 3ema+Channel+Strategy+Backtesting+(c)

In [1]:
#!pip install yfinance

In [2]:
#!pip install cufflinks

In [3]:
import yfinance as yf
import datetime as dt
import pandas as pd 
import numpy as np
import math
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

init_notebook_mode(connected = True)
cf.go_offline()

In [4]:
def calculate_ema(prices, days, smoothing=2):
    ema = [sum(prices[:days]) / days]
    for price in prices[days:]:
        ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
    return ema

In [5]:
def ema3channel_add_trading(ticker, previous_days):
    
    start = dt.datetime.today()-dt.timedelta(previous_days)
    end = dt.datetime.today()
    ohlc_intraday = {}
    ohlc_intraday[ticker] = yf.download(ticker,start,end)
    ohlc_intraday[ticker] = ohlc_intraday[ticker].reset_index(drop = False)
    global max_date 
    global df_Equi_Curve
    if len(ohlc_intraday[ticker]) > max_date :
        max_date = len(ohlc_intraday[ticker])
        df_Equi_Curve['Date'] = ohlc_intraday[ticker]['Date']
    df_signal = ohlc_intraday[ticker].copy()
    df_signal = df_signal[['Date','Open','High','Low','Close']]
    ema50=pd.DataFrame(calculate_ema(df_signal["Close"],50),index=np.arange(50,len(df_signal["Close"])+1),columns=['ema50'])
    ema100=pd.DataFrame(calculate_ema(df_signal["Close"],100),index=np.arange(100,len(df_signal["Close"])+1),columns=['ema100'])
    ema150=pd.DataFrame(calculate_ema(df_signal["Close"],150),index=np.arange(150,len(df_signal["Close"])+1),columns=['ema150'])

    df_signal = pd.concat([df_signal,ema50,ema100,ema150],axis=1,join='outer')[:-1]
    df_signal['Low_t-2']=df_signal['Low'].shift(2)
    df_signal['ema50_t-2']=df_signal['ema50'].shift(2)
    df_signal['Close_t-1']=df_signal['Close'].shift(1)
    df_signal['ema50_t-1']=df_signal['ema50'].shift(1)
    df_signal['ema100_t-1']=df_signal['ema100'].shift(1)
    df_signal['ema150_t-1']=df_signal['ema150'].shift(1)

    condition_t_2 = (df_signal['Low_t-2']<df_signal['ema50_t-2'])
    condition_t_1 = (df_signal['Close_t-1']>df_signal['ema50_t-1']) & (df_signal['ema50_t-1']>df_signal['ema100_t-1']) & (df_signal['ema100_t-1']>df_signal['ema150_t-1'])

    buy_condition = (condition_t_2) & (condition_t_1) 
    df_signal['Buy']=np.where(buy_condition, 'Buy', '')
    df_signal = df_signal[['Date','Open','Close','ema50','ema100','ema150','Buy']]
    
    condition = (df_signal['Buy']=='Buy')
    df_signal2 = df_signal[condition].copy()
    df_signal2 = df_signal2.reset_index(drop = True)
    df_signal2 = df_signal2[['Date','Buy']]
    
    global df_Trading
    
    for i in range(len(df_signal2)):
        date_index = df_signal[df_signal['Date']==df_signal2['Date'][i]].index[0]
        Date_Buy = df_signal['Date'][date_index]
        Price_Buy = df_signal['Open'][date_index]
        stop_loss = 0.92*Price_Buy
        date_index += 1
        stage = 'ToStop'
        close_order = False
        noskip = date_index<len(df_signal)
        while True & noskip:
            if stage == 'ToStop':
                if (df_signal['Close'][date_index] < stop_loss) or (df_signal['Close'][date_index] < df_signal['ema150'][date_index]):
                    close_order = True
                    break
                elif df_signal['Close'][date_index] > Price_Buy*1.05:
                    stage = 'ToProfit'
            elif stage == 'ToProfit':  
                if df_signal['Close'][date_index] < df_signal['ema50'][date_index]:
                    close_order = True
                    break
                elif df_signal['Close'][date_index] < Price_Buy*1.05:
                    stage = 'ToStop'
            date_index += 1
            if date_index >= len(df_signal):
                break
        if close_order &(date_index+1 < len(df_signal)):
            date_index += 1
            Date_Sell = df_signal['Date'][date_index]
            Price_Sell = df_signal['Open'][date_index]
            if np.isnan(Price_Sell):
                Date_Sell = df_signal['Date'][date_index+1]
                Price_Sell = df_signal['Open'][date_index+1]
            fee = 0.002*(Price_Sell+Price_Buy)
            PL = (Price_Sell-Price_Buy-fee)*100/(Price_Buy-fee)
            Holding_Days = abs(Date_Sell-Date_Buy).days
            
        else:
            Date_Sell = np.NaN
            Price_Sell = np.NaN
            PL = np.NaN
            Holding_Days = np.NaN
            
        if noskip:    
            ROC = (Price_Buy - df_signal['Open'][date_index-150])/df_signal['Open'][date_index-150]    
            Add_Trading = pd.DataFrame([[ticker,Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL,Holding_Days,ROC]],columns = columns_Trading)     
            df_Trading = df_Trading.append(Add_Trading)


In [9]:
previous_days = 3000
df = pd.read_excel(r'input/Nasdaq100.xlsx',header=None)
df.columns = ['Nasdaq']
tickers = df['Nasdaq'].values.tolist()

columns_Trading = ['Ticker','Date_Buy','Price_Buy','Date_Sell'
                   ,'Price_Sell','PL(%)','Holding_Days','ROC150']
df_Trading = pd.DataFrame(columns = columns_Trading)

columns_Picking = ['Ticker','Date_Buy','Price_Buy','Amount_Buy','Date_Sell'
                   ,'Price_Sell','Amount_Sell','PL(%)','Amount_Profit','Holding_Days']
df_Picking = pd.DataFrame(columns = columns_Picking)

columns_Equi = ['Date', 'Equity Curve','Stock Holding']
df_Equi_Curve = pd.DataFrame(columns = columns_Equi)

max_date = 0

for ticker in tickers:
     ema3channel_add_trading(ticker, previous_days)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ALXN: No data found, symbol may be delisted


ValueError: Shape of passed values is (1, 1), indices imply (0, 1)

In [10]:
columns_Picking = ['Ticker','Date_Buy','Price_Buy','Amount_Buy','Date_Sell','Price_Sell'
                   ,'Amount_Sell','PL(%)','Amount_Profit','Holding_Days']
df_Picking = pd.DataFrame(columns = columns_Picking)

Port = []
amount = 100
equi_curve = 100
position = 0.025

for i in range(len(df_Equi_Curve)):
    expected_buy = position*equi_curve
    gap = math.floor(amount/expected_buy)
    
    df_buy = df_Trading[df_Trading['Date_Buy'] == df_Equi_Curve['Date'][i]].sort_values(by=['ROC150']
                                                    ,ascending=True).reset_index(drop=True).copy()
    if (len(df_buy)>0) & (gap>0):
        if gap>=len(df_buy):
            num = len(df_buy)        
        else:
            num = gap
        for j in range(num):
            amount -= (expected_buy*1.002)
            Port.append(df_buy['Ticker'][j])
            if not np.isnan(df_buy['Price_Sell'][j]):
                amount_sell = (expected_buy*df_buy['Price_Sell'][j]/df_buy['Price_Buy'][j])*(1-0.002)
                pl = (amount_sell-(expected_buy*1.002))*100/(expected_buy*1.002)
                amount_profit = amount_sell-(expected_buy*1.002)
            else:
                amount_sell = np.NaN
                pl = np.NaN
                amount_profit = np.NaN
            list_add = [df_buy['Ticker'][j],df_buy['Date_Buy'][j],df_buy['Price_Buy'][j],expected_buy*1.002
                        ,df_buy['Date_Sell'][j],df_buy['Price_Sell'][j],amount_sell,pl,amount_profit
                        ,df_buy['Holding_Days'][j]]
            add_picking = pd.DataFrame([list_add],columns = columns_Picking)
            df_Picking  = df_Picking.append(add_picking)
            
    df_sell = df_Picking[df_Picking['Date_Sell'] == df_Equi_Curve['Date'][i]].reset_index(drop=True).copy()
    
    for k in range(len(df_sell)):
        if df_sell['Ticker'][k] in Port:
            Port.remove(df_sell['Ticker'][k])
            amount += df_sell['Amount_Sell'][k]
            equi_curve += df_sell['Amount_Profit'][k]
            
    df_Equi_Curve.loc[df_Equi_Curve['Date']==df_Equi_Curve['Date'][i],'Equity Curve'] =equi_curve  
    df_Equi_Curve.loc[df_Equi_Curve['Date']==df_Equi_Curve['Date'][i],'Stock Holding'] =len(Port)
    

In [11]:
#Trading Summary
max_=0
max_dd=0
for i in range(len(df_Equi_Curve)):
    if df_Equi_Curve['Equity Curve'][i] > max_:
        max_=df_Equi_Curve['Equity Curve'][i]
    else:
        dd = (max_ - df_Equi_Curve['Equity Curve'][i])*100/max_
        if dd > max_dd:
            max_dd = dd

returns = df_Equi_Curve['Equity Curve'][len(df_Equi_Curve)-1]-100
cagr = ((((returns+100)/100)**(1/(len(df_Equi_Curve)/250)))-1)*100
win_trade = df_Picking[df_Picking["PL(%)"]>0].count()["PL(%)"]
loss_trade =  df_Picking[df_Picking["PL(%)"]<=0].count()["PL(%)"]
win_rate = win_trade*100/(win_trade+loss_trade)
avg_gain = df_Picking[df_Picking["PL(%)"]>0]["PL(%)"].mean()
avg_loss = df_Picking[df_Picking["PL(%)"]<=0]["PL(%)"].mean()
expected_return = (win_rate*avg_gain/100)+((1-win_rate)*avg_loss/100)
list_ = [["Total Profit",str(round(returns,2))+'%'],
            ["CAGR",str(round(cagr,2))+'%'],
             ["Max Drawdown",str(round(max_dd,2))+'%'],
             ["Win Trade",win_trade],["Loss Trade ",loss_trade],
             ["Win Rate",str(round(win_rate,2))+'%'],["Average Gain",str(round(avg_gain,2))+'%'],
             ["Average loss",str(round(avg_loss,2))+'%'],["Expected Return",str(round(expected_return,2))+'%']]
                      
df_summary = pd.DataFrame(list_,columns=['Measurement','Result'])
df_summary

,Measurement,Result
0,Total Profit,55.01%
1,CAGR,5.44%
2,Max Drawdown,5.14%
3,Win Trade,227
4,Loss Trade,263
5,Win Rate,46.33%
6,Average Gain,15.53%
7,Average loss,-6.23%
8,Expected Return,10.02%


In [12]:
df_plt = df_Equi_Curve.copy()
df_plt.set_index('Date',inplace = True)
df_plt.index = pd.to_datetime(df_plt.index, format='%d-%m-%Y')
df_Picking = df_Picking.round(2)

fig = make_subplots(rows=2, cols=1,shared_xaxes=True, vertical_spacing=0.02,
         specs=[[{"type": "table"}],
         [{"type": "table"}]])

fig.add_trace(go.Table(header=dict(values=df_summary.columns,font=dict(size=10),align="left"),
         cells=dict(values=[df_summary[k].tolist() for k in df_summary.columns],align = "left")),row=1, col=1)

fig.add_trace(go.Table(header=dict(values=df_Picking.columns,font=dict(size=10),align="left"),
         cells=dict(values=[df_Picking[k].tolist() for k in df_Picking.columns],align = "left")),row=2, col=1)

fig.update_layout( height=600, showlegend=True)

In [13]:
#Plot equity curve
fig = px.area(x=df_plt.index,y= df_plt['Equity Curve'])
fig.update_layout( title_text='Equity Curve')   
fig.show()

In [14]:
fig = px.area(x=df_plt.index,y= df_plt['Stock Holding'])  
fig.update_layout( title_text='Stock Holding')   
fig.show()